## Setup


In [1]:
import os
import sys

from datasets import load_dataset
from loguru import logger
from pydantic import BaseModel

sys.path.insert(0, "..")

from src.sample import InteractionDataSampler

In [2]:
class Args(BaseModel):
    run_name: str = "00-prep-data"
    testing: bool = True
    notebook_persist_dp: str = None
    random_seed: int = 42

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    sample_users: int = 1000
    min_user_interactions: int = 5
    min_item_interactions: int = 10

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        if not self.testing:
            os.makedirs(self.notebook_persist_dp, exist_ok=True)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

{
  "run_name": "00-prep-data",
  "testing": true,
  "notebook_persist_dp": "/home/duong/Documents/datn1/notebooks/data/00-prep-data",
  "random_seed": 42,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "sample_users": 1000,
  "min_user_interactions": 5,
  "min_item_interactions": 10
}


## Load data


In [3]:
dataset = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023",
    "5core_timestamp_Toys_and_Games",
    trust_remote_code=True,
)


def parse_dtype(df):
    return df.assign(
        rating=lambda df: df[args.rating_col].astype(float),
        timestamp=lambda df: df[args.timestamp_col].astype(int),
    )


train_raw = dataset["train"].to_pandas().pipe(parse_dtype)
val_raw = dataset["valid"].to_pandas().pipe(parse_dtype)
test_raw = dataset["test"].to_pandas().pipe(parse_dtype)

In [4]:
import pandas as pd


def print_min_max_time(df, df_name):
    # Lấy min và max timestamp
    min_time = df["timestamp"].min()
    max_time = df["timestamp"].max()

    # Chuyển đổi từ milliseconds sang seconds và thành datetime (UTC)
    min_datetime = pd.to_datetime(min_time, unit="ms").tz_localize("UTC")
    max_datetime = pd.to_datetime(max_time, unit="ms").tz_localize("UTC")

    # Chuyển đổi sang timezone khác (ví dụ: 'US/Pacific')
    min_datetime_pst = min_datetime.tz_convert("US/Pacific")
    max_datetime_pst = max_datetime.tz_convert("US/Pacific")

    print(f"\n{df_name}:")
    print(
        f"- Min timestamp (raw): {min_time} → Datetime (UTC): {min_datetime} → PST: {min_datetime_pst}"
    )
    print(
        f"- Max timestamp (raw): {max_time} → Datetime (UTC): {max_datetime} → PST: {max_datetime_pst}"
    )


# Áp dụng cho từng tập dữ liệu
print_min_max_time(train_raw, "Train")
print_min_max_time(val_raw, "Validation")
print_min_max_time(test_raw, "Test")


Train:
- Min timestamp (raw): 952575870000 → Datetime (UTC): 2000-03-09 04:24:30+00:00 → PST: 2000-03-08 20:24:30-08:00
- Max timestamp (raw): 1628643348297 → Datetime (UTC): 2021-08-11 00:55:48.297000+00:00 → PST: 2021-08-10 17:55:48.297000-07:00

Validation:
- Min timestamp (raw): 1628643494904 → Datetime (UTC): 2021-08-11 00:58:14.904000+00:00 → PST: 2021-08-10 17:58:14.904000-07:00
- Max timestamp (raw): 1658002608846 → Datetime (UTC): 2022-07-16 20:16:48.846000+00:00 → PST: 2022-07-16 13:16:48.846000-07:00

Test:
- Min timestamp (raw): 1658002857227 → Datetime (UTC): 2022-07-16 20:20:57.227000+00:00 → PST: 2022-07-16 13:20:57.227000-07:00
- Max timestamp (raw): 1694444496685 → Datetime (UTC): 2023-09-11 15:01:36.685000+00:00 → PST: 2023-09-11 08:01:36.685000-07:00


In [5]:
train_raw

,user_id,parent_asin,rating,timestamp
0,AGKASBHYZPGTEPO6LWZPVJWB2BVA,B006GBITXC,3.0,1452647382000
1,AGKASBHYZPGTEPO6LWZPVJWB2BVA,B00TLEMSVK,4.0,1454675785000
2,AGKASBHYZPGTEPO6LWZPVJWB2BVA,B00SO7HF6I,3.0,1454676014000
3,AGKASBHYZPGTEPO6LWZPVJWB2BVA,B00MZG6OO8,3.0,1471541996000
4,AGKASBHYZPGTEPO6LWZPVJWB2BVA,B007JWWUDW,5.0,1471542588000
...,...,...,...,...
3114942,AHMDS2PYZIJWE6SBXGDTBSJ4SRLA,B00BY2ER66,1.0,1379154911000
3114943,AHMDS2PYZIJWE6SBXGDTBSJ4SRLA,B00CVDMCH8,3.0,1379155054000
3114944,AHMDS2PYZIJWE6SBXGDTBSJ4SRLA,B00BY2ER6G,5.0,1379155144000
3114945,AHMDS2PYZIJWE6SBXGDTBSJ4SRLA,B00I8Z6GAM,5.0,1420664276000


In [6]:
import datetime

max_timestamp = train_raw["timestamp"].max()
max_datetime = datetime.datetime.fromtimestamp(max_timestamp / 1000)
print("Max datetime:", max_datetime)

Max datetime: 2021-08-11 07:55:48.297000


In [7]:
import datetime

max_timestamp = val_raw["timestamp"].max()
max_datetime = datetime.datetime.fromtimestamp(max_timestamp / 1000)
print("Max datetime:", max_datetime)

Max datetime: 2022-07-17 03:16:48.846000


## Sample

In [8]:
data_sampler = InteractionDataSampler(
    user_col=args.user_col,
    item_col=args.item_col,
    sample_users=args.sample_users,
    min_val_records=1000,
    random_seed=args.random_seed,
    min_item_interactions=args.min_item_interactions,
    min_user_interactions=args.min_user_interactions,
    debug=False,
)
sample_df, val_sample_df = data_sampler.sample(train_raw, val_raw)

2025-06-19 02:02:40.177 | INFO     | src.sample:sample:119 - After randomly removing users - round 1: num_users=407,842
2025-06-19 02:02:40.178 | INFO     | src.sample:sample:123 - Number of users 407,842 are still greater than expected, keep removing...




Randomly removing 2529 users - Round 2 started


2025-06-19 02:02:45.250 | INFO     | src.sample:sample:119 - After randomly removing users - round 2: num_users=252,964
2025-06-19 02:02:45.250 | INFO     | src.sample:sample:123 - Number of users 252,964 are still greater than expected, keep removing...




Randomly removing 2494 users - Round 3 started


2025-06-19 02:02:51.290 | INFO     | src.sample:sample:119 - After randomly removing users - round 3: num_users=249,424
2025-06-19 02:02:51.291 | INFO     | src.sample:sample:123 - Number of users 249,424 are still greater than expected, keep removing...


...

2025-06-19 02:07:52.077 | INFO     | src.sample:sample:119 - After randomly removing users - round 149: num_users=14,037
2025-06-19 02:07:52.078 | INFO     | src.sample:sample:123 - Number of users 14,037 are still greater than expected, keep removing...




Randomly removing 134 users - Round 150 started


2025-06-19 02:07:52.466 | INFO     | src.sample:sample:119 - After randomly removing users - round 150: num_users=13,433
2025-06-19 02:07:52.466 | INFO     | src.sample:sample:123 - Number of users 13,433 are still greater than expected, keep removing...




Randomly removing 128 users - Round 151 started


2025-06-19 02:07:52.759 | INFO     | src.sample:sample:119 - After randomly removing users - round 151: num_users=12,873
2025-06-19 02:07:52.760 | INFO     | src.sample:sample:123 - Number of users 12,873 are still greater than expected, keep removing...
2025-06-19 02:07:52.775 | INFO     | src.sample:sample:140 - Number of val_df records 966 are falling below expected threshold, stop and use `sample_df` as final output...
2025-06-19 02:07:52.781 | INFO     | src.sample:sample:149 - len(sample_users)=12,409 len(sample_items)=4,143


In [9]:
train_sample_df = sample_df.copy()
sample_users = train_sample_df[args.user_col].unique()
sample_items = train_sample_df[args.item_col].unique()
logger.info(f"{len(sample_users)=} {len(sample_items)=}")

val_sample_df = val_raw.loc[
    lambda df: df[args.user_col].isin(sample_users)
    & df[args.item_col].isin(sample_items)
]

train_items = train_sample_df[args.item_col].unique()
train_users = train_sample_df[args.user_col].unique()
val_items = val_sample_df[args.item_col].unique()
val_users = val_sample_df[args.user_col].unique()
logger.info(f"{len(train_items)=}, {len(train_users)=}")
logger.info(f"{len(val_items)=}, {len(val_users)=}")
val_users_in_train = set(val_users).intersection(set(train_users))
val_items_in_train = set(val_items).intersection(set(train_items))
logger.info(
    f"Percentage of val users in train: {len(val_users_in_train) / len(val_users):,.0%}"
)
logger.info(
    f"Percentage of val items in train: {len(val_items_in_train) / len(val_items):,.0%}"
)
logger.info(
    f"Sparsity: {1 - len(train_sample_df) / (len(train_items) * len(train_users)):,.4%}"
)

2025-06-19 02:07:52.793 | INFO     | __main__:<module>:4 - len(sample_users)=12409 len(sample_items)=4143
2025-06-19 02:07:52.818 | INFO     | __main__:<module>:15 - len(train_items)=4143, len(train_users)=12409
2025-06-19 02:07:52.819 | INFO     | __main__:<module>:16 - len(val_items)=651, len(val_users)=644
2025-06-19 02:07:52.820 | INFO     | __main__:<module>:19 - Percentage of val users in train: 100%
2025-06-19 02:07:52.820 | INFO     | __main__:<module>:22 - Percentage of val items in train: 100%
2025-06-19 02:07:52.820 | INFO     | __main__:<module>:25 - Sparsity: 99.8236%


In [10]:
assert train_sample_df.groupby(args.user_col).size().min() >= args.min_user_interactions
assert train_sample_df.groupby(args.item_col).size().min() >= args.min_item_interactions

In [11]:
train_sample_df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
user_id,90700,12409,AGTM6TECFNENFVXRGNVCFUYVL22A,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN
parent_asin,90700,4143,B0BW3QTWJJ,437,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rating,90700.0,NaN,NaN,NaN,4.554344,0.928609,1.0,5.0,5.0,5.0,5.0
timestamp,90700.0,NaN,NaN,NaN,1490914780140.189941,68577577958.464668,1036400058000.0,1442153383000.0,1485965216500.0,1546654323543.5,1628640736369.0


## Persist sample

In [13]:
train_sample_df.to_parquet("../data/train.parquet")
val_sample_df.to_parquet("../data/val.parquet")

In [14]:
from datasets import load_dataset

metadata_raw = load_dataset(
    "McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Toys_and_Games", trust_remote_code=True
)
metadata_raw_df = metadata_raw["full"].to_pandas()
metadata_raw_df.to_parquet("../data/raw_meta.parquet")